# 🧠 Complete Image Preprocessing and Multi-Class Classification using CNN (Keras + CIFAR-10)

This notebook demonstrates a complete pipeline for image preprocessing and classification using a **Convolutional Neural Network (CNN)** on the CIFAR-10 dataset.

In [ ]:

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

print("TensorFlow version:", tf.__version__)


## 1️⃣ Load the CIFAR-10 Dataset

In [ ]:

(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print("Training data shape:", train_images.shape)
print("Testing data shape:", test_images.shape)


## 2️⃣ Visualize Raw Images

In [ ]:

plt.figure(figsize=(8,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(train_images[i])
    plt.title(class_names[train_labels[i][0]])
    plt.axis('off')
plt.show()


## 3️⃣ Normalize Pixel Values (0–1)

In [ ]:

train_images, test_images = train_images / 255.0, test_images / 255.0


## 4️⃣ Data Augmentation

In [ ]:

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])


## 5️⃣ Visualize Augmented Images

In [ ]:

plt.figure(figsize=(8,8))
for i in range(9):
    augmented_image = data_augmentation(tf.expand_dims(train_images[i], 0))
    plt.subplot(3,3,i+1)
    plt.imshow(augmented_image[0])
    plt.title("Augmented")
    plt.axis('off')
plt.show()


## 6️⃣ Define the CNN Model

In [ ]:

model = models.Sequential([
    data_augmentation,
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.summary()


## 7️⃣ Compile the Model

In [ ]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


## 8️⃣ Train the CNN

In [ ]:

history = model.fit(
    train_images, train_labels,
    epochs=15,
    batch_size=64,
    validation_data=(test_images, test_labels),
    verbose=2
)


## 9️⃣ Evaluate the Model

In [ ]:

test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print("Test accuracy:", test_acc)


## 🔟 Visualize Training Accuracy

In [ ]:

plt.figure(figsize=(8,6))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('CNN Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


## 🧩 Predict and Visualize Results

In [ ]:

predictions = model.predict(test_images)
plt.figure(figsize=(8,8))
for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(test_images[i])
    plt.title(f"True: {class_names[test_labels[i][0]]}\nPred: {class_names[np.argmax(predictions[i])]}")
    plt.axis('off')
plt.show()


## ✅ Summary: ANN vs CNN


| Feature | ANN | CNN |
|----------|-----|-----|
| Input shape | Flattened (1D) | 2D spatial (H×W×C) |
| Learns spatial patterns? | ❌ No | ✅ Yes |
| Parameters | High | Lower (shared weights) |
| Use-case | Simple / tabular | Image, spatial data |
